In [1]:
import pandas as pd

Task 1: Create Import file (try, except)

Task 2: Inspect Value (check if has 26 values, print invalid, report)

Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'